In [2]:
# python libraries
import numpy as np
from IPython.display import HTML, display
# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, DirectCollocation, Solve,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat)

from underactuated import running_as_notebook

from pydrake.all import (
    BallRpyJoint,
    Box,
    CoulombFriction,
    Cylinder,
    PrismaticJoint,
    RevoluteJoint,
    RigidTransform,
    SpatialInertia,
    Sphere,
    UnitInertia,
    RotationMatrix,
    JointSliders,
    RollPitchYaw,
    FixedOffsetFrame
)
from pydrake.all import MultibodyPlant, SceneGraph
from underactuated.scenarios import AddShape
import math

In [3]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://37542bb9-cce2-4bf4-8f74-026a18423758.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


## Trajectory Optimization of a Free-Falling Cat

In [4]:
def xyz_rpy_deg(xyz, rpy_deg):
    """Shorthand for defining a pose."""
    rpy_deg = np.asarray(rpy_deg)
    return RigidTransform(RollPitchYaw(rpy_deg * np.pi / 180), xyz)

In [5]:
seg_length = 0.175   # m
seg_diam = 0.15      # m
seg_mass = 1         # kg
segXverse_moi = 1    # kg-m^2
JIratio = 0.25       # (xverse to axial MOI ratio)
# tau_max = 40Nm (max torque)
I_xx = JIratio*segXverse_moi      # segAxialMOI
I_yy = segXverse_moi
I_zz = segXverse_moi
# I_yy = I_zz = segXverseMOI
# I_xy = I_xz = I_yz = 0

# S
# B1
# B2

# continuous mbp
# cat = MultibodyPlant(0.0)
# cat.AddRigidBody()

builder = DiagramBuilder()
cat_plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
# cat_plant = MultibodyPlant(0.0)
# scene_graph = SceneGraph()
# cat_plant.RegisterAsSourceForSceneGraph(scene_graph)

segment1_inertia = SpatialInertia(
                mass=1, p_PScm_E=[0.0,0.0,0.0], G_SP_E=UnitInertia(I_xx, I_yy, I_zz))
segment2_inertia = SpatialInertia(
                mass=1, p_PScm_E=[0.0,0.0,0.0], G_SP_E=UnitInertia(I_xx, I_yy, I_zz))
no_inertia = SpatialInertia(
        mass=0, p_PScm_E=[0.0, 0.0, 0.0], G_SP_E=UnitInertia(0, 0, 0))
segment1_shape = Cylinder(seg_diam/2, seg_length)
segment2_shape = Cylinder(seg_diam/2, seg_length)

s_body = cat_plant.AddRigidBody("S", no_inertia)
B1_body = cat_plant.AddRigidBody("B1", segment1_inertia)
B2_body = cat_plant.AddRigidBody("B2", segment2_inertia)

B1_end_frame = cat_plant.AddFrame(FixedOffsetFrame("B1_end", B1_body.body_frame(), xyz_rpy_deg([-seg_length/2,0,0],[0,0,0])))
B2_end_frame = cat_plant.AddFrame(FixedOffsetFrame("B2_end", B2_body.body_frame(), xyz_rpy_deg([seg_length/2,0,0],[0,0,0])))

cyl1_xform = xyz_rpy_deg([0,0,0], [90,0,90])
cyl2_xform = xyz_rpy_deg([0,0,0], [90,0,-90])
leg1_xform = xyz_rpy_deg([0.0525, 0.05, 0.0866], [-30,0,0])
leg2_xform = xyz_rpy_deg([0.0525, -0.05, 0.0866], [30,0,0])
leg3_xform = xyz_rpy_deg([-0.0525, 0.05, 0.0866], [-30,0,0])
leg4_xform = xyz_rpy_deg([-0.0525, -0.05, 0.0866], [30,0,0])
leg_shape = Box(.02, .02, .1)
# RigidTransform()
B1_geoID = cat_plant.RegisterVisualGeometry(B1_body, cyl1_xform, segment1_shape, "B1_cylinder", [0,1,0,1])
leg1_geoID = cat_plant.RegisterVisualGeometry(B1_body, leg1_xform, leg_shape, "leg1_box",[1,0,0,1])
leg2_geoID = cat_plant.RegisterVisualGeometry(B1_body, leg2_xform, leg_shape, "leg2_box",[1,0,0,1])
B2_geoID = cat_plant.RegisterVisualGeometry(B2_body, cyl2_xform, segment2_shape, "B2_cylinder", [0,1,0,1])
leg3_geoID = cat_plant.RegisterVisualGeometry(B2_body, leg3_xform, leg_shape, "leg3_box",[1,0,0,1])
leg4_geoID = cat_plant.RegisterVisualGeometry(B2_body, leg4_xform, leg_shape, "leg4_box",[1,0,0,1])

s_world_zjoint = PrismaticJoint("z", cat_plant.world_frame(), s_body.body_frame(), [0,0,1])
B1_s_joint = BallRpyJoint("B1_rpy", s_body.body_frame(), B1_end_frame)
B2_s_joint = BallRpyJoint("B2_rpy", s_body.body_frame(), B2_end_frame)
cat_plant.AddJoint(s_world_zjoint)
# B1_s_actuator = cat_plant.AddJointActuator("B1_rpy_act", B1_s_joint)
# B2_s_actuator = cat_plant.AddJointActuator("B2_rpy_act", B2_s_joint)
# cat_plant.AddJoint(s_world_zjoint)
cat_plant.AddJoint(B1_s_joint)
cat_plant.AddJoint(B2_s_joint)

# uncomment this if you want to use jointsliders to test below
meshcat_vis = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

cat_plant.Finalize()
cat_plant.set_name("my_cat")

# print(cat_plant.CreateDefaultContext())
# print(cat_plant.get_actuation_input_port())
print(cat_plant.num_positions())
print(cat_plant.num_velocities())
print(cat_plant.num_actuated_dofs())
print(cat_plant.get_actuation_input_port().get_index())

# my_cat = builder.AddSystem(cat_plant)
# scene_graph = builder.AddSystem(SceneGraph())

7
7
0
InputPortIndex(3)


In [19]:
# seg_length = 0.175   # m
# seg_diam = 0.15      # m
# seg_mass = 1         # kg
# segXverse_moi = 1    # kg-m^2
# JIratio = 0.25       # (xverse to axial MOI ratio)
# # tau_max = 40Nm (max torque)
# I_xx = JIratio*segXverse_moi      # segAxialMOI
# I_yy = segXverse_moi
# I_zz = segXverse_moi
# # I_yy = I_zz = segXverseMOI
# # I_xy = I_xz = I_yz = 0

# # S
# # B1
# # B2

# # continuous mbp
# # cat = MultibodyPlant(0.0)
# # cat.AddRigidBody()

# builder = DiagramBuilder()
# cat_plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
# # cat_plant = MultibodyPlant(0.0)

# segment1_inertia = SpatialInertia(
#                 mass=1, p_PScm_E=[0.0,0.0,0.0], G_SP_E=UnitInertia(I_xx, I_yy, I_zz))
# segment2_inertia = SpatialInertia(
#                 mass=1, p_PScm_E=[0.0,0.0,0.0], G_SP_E=UnitInertia(I_xx, I_yy, I_zz))
# no_inertia = SpatialInertia(
#         mass=0, p_PScm_E=[0.0, 0.0, 0.0], G_SP_E=UnitInertia(0, 0, 0))
# segment1_shape = Cylinder(seg_diam/2, seg_length)
# segment2_shape = Cylinder(seg_diam/2, seg_length)

# s_body = cat_plant.AddRigidBody("S", no_inertia)
# B1_body = cat_plant.AddRigidBody("B1", segment1_inertia)
# B2_body = cat_plant.AddRigidBody("B2", segment2_inertia)
# # B1_body = AddRigidBody(cat_plant, "B1", segment1_inertia)
# # B2_body = AddRigidBody(cat_plant, "B2", segment2_inertia)

# B1_transform = RigidTransform(RotationMatrix(), [seg_length/2, 0, 0])
# B2_transform = RigidTransform(RotationMatrix(np.array([[-1,0,0],[0,-1,0],[0,0,1]])), [-seg_length/2, 0, 0])

# # B1_geoID = AddShape(cat_plant, segment_shape, "B1_cylinder")
# cyl1_xform = xyz_rpy_deg([seg_length/2,0,0], [90,0,90])
# cyl2_xform = xyz_rpy_deg([-seg_length/2,0,0], [90,0,-90])
# # leg1_xform = xyz_rpy_deg([0.0866, -0.05, -0.14], [-30,0,0])
# # leg1_xform = xyz_rpy_deg([0.0866, -0.05, -0.14], [0,0,0])
# leg1_xform = xyz_rpy_deg([0.14, 0.05, 0.0866], [-30,0,0])
# leg2_xform = xyz_rpy_deg([0.14, -0.05, 0.0866], [30,0,0])
# leg3_xform = xyz_rpy_deg([-0.14, 0.05, 0.0866], [-30,0,0])
# leg4_xform = xyz_rpy_deg([-0.14, -0.05, 0.0866], [30,0,0])
# leg_shape = Box(.02, .02, .1)
# # RigidTransform()
# B1_geoID = cat_plant.RegisterVisualGeometry(B1_body, cyl1_xform, segment1_shape, "B1_cylinder", [0,1,0,1])
# leg1_geoID = cat_plant.RegisterVisualGeometry(B1_body, leg1_xform, leg_shape, "leg1_box",[1,0,0,1])
# leg2_geoID = cat_plant.RegisterVisualGeometry(B1_body, leg2_xform, leg_shape, "leg2_box",[1,0,0,1])
# B2_geoID = cat_plant.RegisterVisualGeometry(B2_body, cyl2_xform, segment2_shape, "B2_cylinder", [0,1,0,1])
# leg3_geoID = cat_plant.RegisterVisualGeometry(B2_body, leg3_xform, leg_shape, "leg3_box",[1,0,0,1])
# leg4_geoID = cat_plant.RegisterVisualGeometry(B2_body, leg4_xform, leg_shape, "leg4_box",[1,0,0,1])

# s_world_zjoint = PrismaticJoint("z", cat_plant.world_frame(), s_body.body_frame(), [0,0,1])
# # B1_world_zjoint = PrismaticJoint("z", cat_plant.world_frame(), B1_body.body_frame(), [0,0,1])
# B1_s_joint = BallRpyJoint("B1_rpy", s_body.body_frame(), B1_body.body_frame())
# B2_s_joint = BallRpyJoint("B2_rpy", s_body.body_frame(), B2_body.body_frame())
# # B2_testjoint = PrismaticJoint("x", B1_body.body_frame(), B2_body.body_frame(), [0,0,1])
# cat_plant.AddJoint(s_world_zjoint)
# cat_plant.AddJoint(B1_s_joint)
# cat_plant.AddJoint(B2_s_joint)
# # cat_plant.AddJoint(B2_testjoint)

# meshcat_vis = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# cat_plant.Finalize()
# cat_plant.set_name("my_cat")

# # for reference
# # plant.AddJoint(
# #         PrismaticJoint(
# #             "x", plant.world_frame(), x_body.body_frame(), [1, 0, 0]
# #         )
# #     )
# #  plant.RegisterVisualGeometry(
# #             body, RigidTransform(), shape, name, color
# #         )
# # inertia = SpatialInertia(
# #         mass=0, p_PScm_E=[0.0, 0.0, 0.0], G_SP_E=UnitInertia(0, 0, 0)
# #     )
# #     x_body = plant.AddRigidBody("x", instance, inertia)

In [6]:
# # Test model with joint sliders
meshcat.Delete()
meshcat.DeleteAddedControls()
default_interactive_timeout = None if running_as_notebook else 1.0
sliders = builder.AddSystem(JointSliders(meshcat, cat_plant))
diagram = builder.Build()
sliders.Run(diagram, default_interactive_timeout)
meshcat.Delete()
meshcat.DeleteAddedControls()

INFO:drake:Press the 'Stop JointSliders' button in Meshcat or press 'Escape' to continue.


In [5]:
# meshcat_vis = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# meshcat.Delete()

# diagram = builder.Build()
# context = cat_plant.CreateDefaultContext()
context = cat_plant.CreateDefaultContext()
# diagram.ForcedPublish(context)

x_traj = None
u_traj = None

for N in [25,41]:
    dircol = DirectCollocation(
            cat_plant,
            cat_plant.CreateDefaultContext(),
            num_time_samples=N,
            minimum_timestep=0.5/N,
            maximum_timestep=2.0/N,
        )


            # input_port_index = cat_plant.get_applied_spatial_force_input_port().get_index(),
            # input_port_index=cat_plant.get_actuation_input_port().get_index(),

    prog = dircol.prog()
    dircol.AddEqualTimeIntervalsConstraints()

    x0 = (6.0, 0.0, -np.pi/6, 0.0, 0.0, np.pi/6, 0.0)
    prog.AddBoundingBoxConstraint(
        x0, x0, dircol.initial_state()
    )
    context.SetContinuousState(x0[:])
    diagram.ForcedPublish(context)

    xf = (0.0, np.pi, 0.0, 0.0, np.pi, 0.0, 0.0)
    prog.AddBoundingBoxConstraint(
        xf, xf, dircol.final_state()
    )

    # State constraints hopefully?
    s = dircol.state()
    dircol.AddConstraintToAllKnotPoints(s[3] == 0)
    dircol.AddConstraintToAllKnotPoints(s[6] == 0)
    dircol.AddConstraintToAllKnotPoints(-s[2] == s[5])
    dircol.AddConstraintToAllKnotPoints(s[1] == s[4])

    u = dircol.input()
    dircol.AddRunningCost(u**2)
    dircol.AddFinalCost(dircol.time())

    max_torque = 100.0    # N-m 40 maybe?
    # no fucking clue here
    # dircol.AddConstraintToAllKnotPoints(u[3] == 0)
    # dircol.AddConstraintToAllKnotPoints(u[6] == 0)
    # dircol.AddConstraintToAllKnotPoints(u[1] == u[4])
    # dircol.AddConstraintToAllKnotPoints(-u[2] == u[5])
    dircol.AddConstraintToAllKnotPoints(u[1] <= max_torque)
    dircol.AddConstraintToAllKnotPoints(-max_torque <= u[1])
    dircol.AddConstraintToAllKnotPoints(u[2] <= max_torque)
    dircol.AddConstraintToAllKnotPoints(-max_torque <= u[2])
    dircol.AddConstraintToAllKnotPoints(u[4] <= max_torque)
    dircol.AddConstraintToAllKnotPoints(-max_torque <= u[4])
    dircol.AddConstraintToAllKnotPoints(u[5] <= max_torque)
    dircol.AddConstraintToAllKnotPoints(-max_torque <= u[5])

    if x_traj and u_traj:
        dircol.SetInitialTrajectory(u_traj, x_traj)
    else:
        initial_x_traj = PiecewisePolynomial.FirstOrderHold(
            [0.0, 0.8], np.column_stack((x0, xf))
        )
        dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_traj)

    result = Solve(prog)
    assert result.is_success()

    x_traj = dircol.ReconstructStateTrajectory(result)
    x_traj = dircol.ReconstructInputTrajectory(result)

builder = DiagramBuilder()
source = builder.AddSystem(TrajectorySource(x_traj))
builder.AddSystem(scene_graph)
pos_to_pose = builder.AddSystem(
    MultibodyPositionToGeometryPose(cat_plant, input_multibody_state=True)
)
builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
builder.Connect(
    pos_to_pose.get_output_port(),
    scene_graph.get_source_pose_port(plant.get_source_id())
)

visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(
        scene_graph, xlim=[-2, 2], ylim=[-1.25, 2], show=False
    )
)
builder.Connect(
    scene_graph.get_query_output_port(), visualizer.get_input_port(0)
)
simulator = Simulator(builder.Build())

AdvanceToAndVisualize(
    simulator,
    visualizer,
    x_trajectory.end_time() if running_as_notebook else 0.1,
)

# visualizer.StartRecording()
# for t in np.hstack(
#     (
#         np.arange(x_traj.start_time(), x_traj.end_time(), 1.0 / 32.0),
#         x_traj.end_time(),
#     )
# ):
#     context.SetTime(t)
#     context.SetContinuousState(x_traj.value(t))
#     diagram.ForcedPublish(context)

# visualizer.StopRecording()
# visualizer.PublishRecording()

# for reference
# mbp_x = rng.random((12,))
# mbp_plant.get_input_port().FixValue(mbp_plant_context, u)
# mbp_context.SetContinuousState(mbp_x)
# s0 = 

RuntimeError: The specified input port is abstract-valued, but DirectCollocation only supports vector-valued input ports.  Did you perhaps forget to pass a non-default `input_port_index` argument?

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37542bb9-cce2-4bf4-8f74-026a18423758' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>